In [1]:
%load_ext autoreload

In [71]:
%autoreload 2
from collections import defaultdict
from datetime import datetime
import h5py
from kneed import KneeLocator
import matplotlib.pyplot as plt
from neural_analysis.matIO import loadmat
import numpy as np
import os
import pandas as pd
import re
from scipy.signal import butter, filtfilt, freqz
import sys
import time
from tqdm.auto import tqdm

sys.path.append('../../..')
from ld_utils import compute_eigs, get_data_sigma_norms
from up_down import get_up_down
from utils import compile_folder, get_data_class, get_result_path, load, run_window_selection, save, to_string

# Testing

In [73]:
# session = 'MrJones-Anesthesia-20160107-01'
session = 'Mary-Anesthesia-20160809-01'

pred_steps = 10
pct_of_value = 0.95

In [75]:
VAR_results, window_selection_info = run_window_selection(session, pred_steps=pred_steps, pct_of_value=pct_of_value)

Session Mary-Anesthesia-20160809-01: selected windows by phase with 10-step prediction not found. Running now.
Loading data ...
Data loaded (took 124.07 seconds)
Now computing window = 0.15


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/47035 [00:00<?, ?it/s]

Now computing window = 0.25


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/705 [00:00<?, ?it/s]

  0%|          | 0/705 [00:00<?, ?it/s]

  0%|          | 0/705 [00:00<?, ?it/s]

  0%|          | 0/705 [00:00<?, ?it/s]

KeyError: 0.15